In [3]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

import urllib.request

url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
filename = "hdma-wi-2021.csv"

urllib.request.urlretrieve(url, filename)

# Remove the files if they already exist in HDFS
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

!HADOOP_user=hdfs hdfs dfs -D dfs.replication=1 -put hdma-wi-2021.csv hdfs://boss:9000/single.csv
!HADOOP_user=hdfs hdfs dfs -D dfs.replication=2 -put hdma-wi-2021.csv hdfs://boss:9000/double.csv

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 19337872546 (18.01 GB)
DFS Remaining: 18985226240 (17.68 GB)
DFS Used: 352646306 (336.31 MB)
DFS Used%: 1.82%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.4:9866 (project-4-junho_4-dn-1.project-4-junho_4_default)
Hostname: 35ed3521b189
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 352646306 (336.31 MB)
Non DFS Used: 6466403166 (6.02 GB)
DFS Remaining: 18985226240 (17.68 GB)
DFS Used%: 1.37%
DFS Remaining

In [ ]:
import time
time.sleep(30)

In [4]:
#q9
import requests
import json

# WebHDFS URL for the file
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS&offset=0&length=0"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Initialize an empty dictionary to store the block distribution
block_distribution = {}

# Loop through each block
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    container_id = block['hosts'][0]
    
    # If the status code is 403, use "lost" as the key for your count
    if response.status_code == 403:
        if 'lost' in block_distribution:
            block_distribution['lost'] += 1
        else:
            block_distribution['lost'] = 1
    # Otherwise, count as normal
    else:
        if container_id in block_distribution:
            block_distribution[container_id] += 1
        else:
            block_distribution[container_id] = 1

# Print the block distribution
block_distribution

{'35ed3521b189': 1}

In [5]:
#q10
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS&offset=0&length=0"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Initialize the count of "Single Family"
count = 0

# Loop through each block
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    container_id = block['hosts'][0]
    
    # If the status code is not 403, it's a good block
    if response.status_code != 403:
        # WebHDFS URL for the block
        block_url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={block['offset']}&length={block['length']}&noredirect=true"
        
        # Send a GET request to the block URL
        block_response = requests.get(block_url)
        
        # If the status code is 200, it's a successful request
        if block_response.status_code == 200:
            # Convert the response to JSON
            block_data = block_response.json()
            
            # Get the redirect URL from the data
            redirect_url = block_data['Location']
            
            # Send a GET request to the redirect URL and get the content of the block
            content = requests.get(redirect_url).content.decode('utf-8')
            
            # Count the occurrences of "Single Family" in the content
            count += content.count("Single Family")

count

340847